In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
import torch

# set device
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
import transformers

# set to only report critical errors to avoid excessing logging
transformers.utils.logging.set_verbosity(50)

In [4]:
from nlpsig_networks.scripts.fine_tune_bert_classification import (
    fine_tune_transformer_average_seed,
)

In [5]:
output_dir = "rumours_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## Rumours

In [6]:
%run load_sbert-embeddings.py

In [7]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


## Baseline: Fine-tine BERT for classification

In [8]:
num_epochs = 5
seeds = [1, 12, 123]
validation_metric = "f1"

In [9]:
kwargs = {
    "num_epochs": num_epochs,
    "pretrained_model_name": "bert-base-uncased",
    "df": df_rumours,
    "feature_name": "text",
    "label_column": "label",
    "seeds": seeds,
    "split_ids": split_ids,
    "k_fold": True,
    "validation_metric": validation_metric,
    "device": device,
    "verbose": False,
}

## Focal Loss

In [10]:
loss = "focal"
gamma = 2

In [11]:
bert_classifier = fine_tune_transformer_average_seed(
    loss=loss,
    gamma=gamma,
    results_output=f"{output_dir}/bert_classifier_focal.csv",
    **kwargs,
)

Map:   0%|          | 0/5568 [00:00<?, ? examples/s]

Map:   0%|          | 0/5568 [00:00<?, ? examples/s]

/bask/projects/v/vjgo8416-rough-paths/envs/nlpsig-networks/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.201621,0.710611,0.706456
2,No log,0.168554,0.773633,0.769138
3,No log,0.179551,0.787781,0.778088
4,No log,0.222820,0.783923,0.771469
5,No log,0.308203,0.798714,0.784298


  0%|          | 0/856 [00:00<?, ?it/s]

...

Map:   0%|          | 0/5568 [00:00<?, ? examples/s]

/bask/projects/v/vjgo8416-rough-paths/envs/nlpsig-networks/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.218569,0.644034,0.643380
2,No log,0.180713,0.785761,0.779174
3,No log,0.210476,0.725775,0.725593
4,No log,0.306362,0.760712,0.745402
5,No log,0.349985,0.774555,0.764261


  0%|          | 0/973 [00:00<?, ?it/s]

saving the results dataframe to CSV in rumours_output/bert_classifier_focal.csv


In [12]:
bert_classifier

,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,seed,loss,gamma,k_fold
0,0.628705,0.589027,"[0.7167235494880547, 0.46133044889129254]",0.589742,"[0.7110609480812641, 0.4684239428885228]",0.588463,"[0.7224770642201835, 0.45444858817261585]",1,focal,2,True
0,0.624231,0.579203,"[0.7168539325842697, 0.44155124653739614]",0.581270,"[0.7026431718061674, 0.4598961338718984]",0.578133,"[0.731651376146789, 0.4246137453383058]",12,focal,2,True
0,0.609506,0.559226,"[0.7080953044447542, 0.4103574444131719]",0.561884,"[0.6888045540796964, 0.434964200477327]",0.558442,"[0.7284977064220184, 0.38838572189664355]",123,focal,2,True


In [13]:
bert_classifier["f1"].mean()

0.5758186543931565

In [14]:
bert_classifier["precision"].mean()

0.5776321585341461

In [15]:
bert_classifier["recall"].mean()

0.5750123670327594

In [16]:
np.stack(bert_classifier["f1_scores"]).mean(axis=0)

array([0.71389093, 0.43774638])

In [17]:
np.stack(bert_classifier["precision_scores"]).mean(axis=0)

array([0.70083622, 0.45442809])

In [18]:
np.stack(bert_classifier["recall_scores"]).mean(axis=0)

array([0.72754205, 0.42248269])

## Using Cross-Entropy loss

In [19]:
loss = "cross_entropy"
gamma = None

In [20]:
bert_classifier_ce = fine_tune_transformer_average_seed(
    loss=loss,
    gamma=gamma,
    results_output=f"{output_dir}/bert_classifier_ce.csv",
    **kwargs,
)

Map:   0%|          | 0/5568 [00:00<?, ? examples/s]

Map:   0%|          | 0/5568 [00:00<?, ? examples/s]

/bask/projects/v/vjgo8416-rough-paths/envs/nlpsig-networks/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.527420,0.757556,0.742670
2,No log,0.460328,0.796141,0.781541
3,No log,0.502343,0.799357,0.774638
4,No log,0.555550,0.801286,0.786902
5,No log,0.584539,0.801929,0.789279


  0%|          | 0/856 [00:00<?, ?it/s]

...

Map:   0%|          | 0/5568 [00:00<?, ? examples/s]

/bask/projects/v/vjgo8416-rough-paths/envs/nlpsig-networks/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.638944,0.666447,0.573224
2,No log,0.518650,0.757416,0.748669
3,No log,0.567206,0.743573,0.740688
4,No log,0.618750,0.766645,0.754072
5,No log,0.695375,0.771259,0.756791


  0%|          | 0/973 [00:00<?, ?it/s]

saving the results dataframe to CSV in rumours_output/bert_classifier_ce.csv


In [21]:
bert_classifier_ce

,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,seed,loss,gamma,k_fold
0,0.638397,0.601872,"[0.7224606580829757, 0.48128342245989303]",0.602054,"[0.7210165619645916, 0.4830917874396135]",0.601700,"[0.7239105504587156, 0.47948854555141185]",1,cross_entropy,None,True
0,0.638397,0.562956,"[0.7445351593363181, 0.3813775510204081]",0.581742,"[0.6885046273745737, 0.4749801429706116]",0.564543,"[0.810493119266055, 0.3185935002663825]",12,cross_entropy,None,True
0,0.635042,0.598378,"[0.7197251646149442, 0.4770299145299145]",0.598502,"[0.7186963979416809, 0.47830744509908946]",0.598258,"[0.720756880733945, 0.4757591901971231]",123,cross_entropy,None,True


In [22]:
bert_classifier_ce["f1"].mean()

0.5877353116740756

In [23]:
bert_classifier_ce["precision"].mean()

0.59409949379836

In [24]:
bert_classifier_ce["recall"].mean()

0.5881669644122721

In [25]:
np.stack(bert_classifier_ce["f1_scores"]).mean(axis=0)

array([0.72890699, 0.44656363])

In [26]:
np.stack(bert_classifier_ce["precision_scores"]).mean(axis=0)

array([0.70940586, 0.47879313])

In [27]:
np.stack(bert_classifier_ce["recall_scores"]).mean(axis=0)

array([0.75172018, 0.42461375])